# SaaS Retention Analytics
## Notebook 01 — Build Customer Features

This notebook prepares a customer-level dataset by combining:
- customers
- subscriptions
- transactions
- user_activity

Output:
customer_features.csv

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)

In [2]:
customers = pd.read_csv("../data/raw/customers.csv")
subscriptions = pd.read_csv("../data/raw/subscriptions.csv")
transactions = pd.read_csv("../data/raw/transactions.csv")
user_activity = pd.read_csv("../data/raw/user_activity.csv")

print(customers.shape)
print(subscriptions.shape)
print(transactions.shape)
print(user_activity.shape)

(50000, 5)
(100000, 6)
(500000, 4)
(1000000, 4)


In [3]:
customers["signup_date"] = pd.to_datetime(customers["signup_date"], errors="coerce")
subscriptions["start_date"] = pd.to_datetime(subscriptions["start_date"], errors="coerce")
subscriptions["end_date"] = pd.to_datetime(subscriptions["end_date"], errors="coerce")
transactions["transaction_date"] = pd.to_datetime(transactions["transaction_date"], errors="coerce")
user_activity["event_date"] = pd.to_datetime(user_activity["event_date"], errors="coerce")

In [4]:
customers.head()

,customer_id,name,email,signup_date,country
0,1,Tara Ramos,pinedajill@gmail.com,2025-02-09,Bermuda
1,2,Casey Sanders,kimberly26@white-velasquez.biz,2022-10-11,Lao People's Democratic Republic
2,3,Amanda Fernandez,annapugh@wiggins.biz,2023-04-19,Sweden
3,4,Manuel Atkins,webbdavid@hotmail.com,2024-05-05,Costa Rica
4,5,Mark Jennings,austin86@hotmail.com,2023-12-25,French Guiana


In [5]:
last_payment = (
    transactions
    .groupby("customer_id", as_index=False)
    .agg(
        last_payment_date=("transaction_date", "max"),
        total_revenue=("amount", "sum"),
        num_transactions=("transaction_id", "count")
    )
)

last_payment.head()

,customer_id,last_payment_date,total_revenue,num_transactions
0,1,2024-10-11,810.32,10
1,2,2024-08-19,1075.64,9
2,3,2025-02-06,558.47,7
3,4,2024-12-21,671.55,8
4,5,2024-04-06,325.84,5


In [6]:
last_login = (
    user_activity[user_activity["event_type"] == "Login"]
    .groupby("customer_id", as_index=False)
    .agg(
        last_login_date=("event_date", "max"),
        total_logins=("event_type", "count")
    )
)

last_login.head()

,customer_id,last_login_date,total_logins
0,1,2024-02-02,10
1,2,2024-05-28,10
2,3,2025-02-21,8
3,4,2024-03-19,2
4,5,2024-03-01,5


In [7]:
subscription_info = (
    subscriptions
    .groupby("customer_id", as_index=False)
    .agg(
        plan_type=("plan_type", "last"),
        subscription_status=("status", "last"),
        subscription_start=("start_date", "min")
    )
)

subscription_info.head()

,customer_id,plan_type,subscription_status,subscription_start
0,1,Basic,Active,2023-12-12
1,4,Premium,Canceled,2022-05-09
2,5,Basic,Active,2024-01-28
3,6,Basic,Canceled,2023-03-02
4,7,Standard,Active,2022-11-09


In [8]:
customer_features = (
    customers
    .merge(subscription_info, on="customer_id", how="left")
    .merge(last_payment, on="customer_id", how="left")
    .merge(last_login, on="customer_id", how="left")
)

customer_features.head()

,customer_id,name,email,signup_date,country,plan_type,subscription_status,subscription_start,last_payment_date,total_revenue,num_transactions,last_login_date,total_logins
0,1,Tara Ramos,pinedajill@gmail.com,2025-02-09,Bermuda,Basic,Active,2023-12-12,2024-10-11,810.32,10.0,2024-02-02,10.0
1,2,Casey Sanders,kimberly26@white-velasquez.biz,2022-10-11,Lao People's Democratic Republic,NaN,NaN,NaT,2024-08-19,1075.64,9.0,2024-05-28,10.0
2,3,Amanda Fernandez,annapugh@wiggins.biz,2023-04-19,Sweden,NaN,NaN,NaT,2025-02-06,558.47,7.0,2025-02-21,8.0
3,4,Manuel Atkins,webbdavid@hotmail.com,2024-05-05,Costa Rica,Premium,Canceled,2022-05-09,2024-12-21,671.55,8.0,2024-03-19,2.0
4,5,Mark Jennings,austin86@hotmail.com,2023-12-25,French Guiana,Basic,Active,2024-01-28,2024-04-06,325.84,5.0,2024-03-01,5.0


In [9]:
customer_features.to_csv("../data/processed/customer_features.csv", index=False)

In [10]:
AS_OF_DATE = transactions["transaction_date"].max()
CHURN_DAYS = 90

AS_OF_DATE, CHURN_DAYS

(Timestamp('2025-03-09 00:00:00'), 90)

In [11]:
customer_features["days_since_payment"] = (
    AS_OF_DATE - customer_features["last_payment_date"]
).dt.days

customer_features["days_since_login"] = (
    AS_OF_DATE - customer_features["last_login_date"]
).dt.days

customer_features[[
    "customer_id",
    "days_since_payment",
    "days_since_login"
]].head()

,customer_id,days_since_payment,days_since_login
0,1,149.0,401.0
1,2,202.0,285.0
2,3,31.0,16.0
3,4,78.0,355.0
4,5,337.0,373.0


In [12]:
customer_features["financial_churn"] = customer_features["days_since_payment"] > CHURN_DAYS
customer_features["engagement_churn"] = customer_features["days_since_login"] > CHURN_DAYS

customer_features["silent_churn"] = (
    (customer_features["days_since_payment"] <= CHURN_DAYS) &
    (customer_features["days_since_login"] > CHURN_DAYS)
)

customer_features["active"] = (
    (customer_features["days_since_payment"] <= CHURN_DAYS) &
    (customer_features["days_since_login"] <= CHURN_DAYS)
)

In [13]:
summary = {
    "Total Customers": customer_features["customer_id"].nunique(),
    "Financial Churn": int(customer_features["financial_churn"].sum()),
    "Engagement Churn": int(customer_features["engagement_churn"].sum()),
    "Silent Churn": int(customer_features["silent_churn"].sum()),
    "Active Customers": int(customer_features["active"].sum())
}

summary

{'Total Customers': 50000,
 'Financial Churn': 21645,
 'Engagement Churn': 25692,
 'Silent Churn': 14729,
 'Active Customers': 13612}

In [14]:
customer_features.to_csv("../data/processed/customer_features.csv", index=False)